In [1]:
!pip install --upgrade datasets torch transformers accelerate bitsandbytes peft -qqq

In [2]:
from datasets import load_dataset
dataset = load_dataset('lmqg/qg_koquad')

README.md:   0%|          | 0.00/6.53k [00:00<?, ?B/s]

qg_koquad.py:   0%|          | 0.00/3.29k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54556 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5766 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5766 [00:00<?, ? examples/s]

In [33]:
test_dataset = dataset['test']
train_dataset = dataset['train'].select(range(1000))  # 데이터셋 유지
val_dataset = dataset['validation']

In [36]:
train_dataset[0].keys()

dict_keys(['answer', 'paragraph_question', 'question', 'sentence', 'paragraph', 'sentence_answer', 'paragraph_answer', 'paragraph_sentence', 'paragraph_id'])

In [37]:
# 토크나이져 polyglot-ko-1.3b  gpt계열의 생성형
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b")
tokenizer.pad_token = tokenizer.eos_token

In [38]:
def preprocess(examples):
    text = \
    f"""paragraph:{examples['paragraph']}    
    question:{examples['question']}    
    answer:{examples['answer']}"""
    return tokenizer(text, padding='max_length', truncation=True, max_length=512)        
remove_columns = train_dataset[0].keys()
tokenized_datasets = train_dataset.map(preprocess,remove_columns = remove_columns)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [39]:
# 일반 fine_tuning
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
from transformers import Trainer,TrainingArguments
training_args = TrainingArguments(
    output_dir = './polyglot_model2',
    overwrite_output_dir = True,
    num_train_epochs = 1,    
    per_device_train_batch_size = 2,
    save_steps = 1000,
    save_total_limit = 2,
)
# 입력 텍스트를 토크나이져한후 모델이 사용할수 있도록 동적 패딩을 적용
# maksed language modeling 을 위해서 일부 토큰을 [MASK]로 변경 (BERT같은 모델에) mlm=Ture
# mlm=False GPT 같은 생성모델
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(    
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = tokenized_datasets
)
trainer.train()

Step,Training Loss
500,2.183900


TrainOutput(global_step=500, training_loss=2.18390087890625, metrics={'train_runtime': 212.2002, 'train_samples_per_second': 4.713, 'train_steps_per_second': 2.356, 'total_flos': 3902074257408000.0, 'train_loss': 2.18390087890625, 'epoch': 1.0})

In [42]:
paragraph =  '천상의 모후의 성경상의 출처는 신약성경에서 찾아볼 수 있다. 마리아는 임금인 메시아의 어머니이다. 루카 복음서 1장 32절에서 대천사 가브리엘은 마리아에게 예수의 잉태를 예고하면서 그가 “큰 인물이 되시고 지극히 높으신 분의 아드님이라 불리실 것이다. 주 하느님께서 그분의 조상 다윗의 왕좌를 그분께 주시어, 그분께서 야곱 집안을 영원히 다스리시리니 그분의 나라는 끝이 없을 것이다.”라고 말하였다. 이 말에 따라 기독교에서는 다윗 임금의 왕위가 예수에게로 계승된 것으로 보고 있다. 고대 이스라엘의 역사를 보면 임금의 어머니를 모후(혹은 여왕)라고 부르며 모셨다. 그리하여 예수가 다윗과 솔로몬의 혈통을 이어받은 천상의 임금이므로, 그 어머니인 마리아는 천상의 모후가 되는 셈이다.',
question =  '역사에서 임금의 어머니를 모후 혹은 여왕이라고 부르며 모신것은?'

In [50]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_text =  f"""paragraph:{paragraph}    
    question:{question}    
    answer:"""
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to(device)

max_length = 256
model = model.to(device)

test_output = model.generate(
    input_ids,
    do_sample=True,
    # max_length=max_length,
    temperature = 0.2,
    max_new_tokens = 128
)
print(tokenizer.decode(test_output.squeeze(),skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


paragraph:('천상의 모후의 성경상의 출처는 신약성경에서 찾아볼 수 있다. 마리아는 임금인 메시아의 어머니이다. 루카 복음서 1장 32절에서 대천사 가브리엘은 마리아에게 예수의 잉태를 예고하면서 그가 “큰 인물이 되시고 지극히 높으신 분의 아드님이라 불리실 것이다. 주 하느님께서 그분의 조상 다윗의 왕좌를 그분께 주시어, 그분께서 야곱 집안을 영원히 다스리시리니 그분의 나라는 끝이 없을 것이다.”라고 말하였다. 이 말에 따라 기독교에서는 다윗 임금의 왕위가 예수에게로 계승된 것으로 보고 있다. 고대 이스라엘의 역사를 보면 임금의 어머니를 모후(혹은 여왕)라고 부르며 모셨다. 그리하여 예수가 다윗과 솔로몬의 혈통을 이어받은 천상의 임금이므로, 그 어머니인 마리아는 천상의 모후가 되는 셈이다.',)    
    question:역사에서 임금의 어머니를 모후 혹은 여왕이라고 부르며 모신것은?    
    answer:고대 이스라엘의 역사를 보면 임금의 어머니를 모후라고 부르며 모셨다.    
    answer:예수가 다윗과 솔로몬의 혈통을 이어받은 천상의 임금이므로, 그 어머니인 마리아는 천상의 모후가 되는 셈이다.    
    answer:고대 이스라엘의 역사를 보면 임금의 어머니를 모후라고 부르며 모셨다.    
    answer:예수가 다윗과 솔로몬의 혈통을 이어받은
